In [8]:
import numpy as np
import pandas as pd
from time import time
import dask.dataframe as dd
from sklearn.cluster import KMeans
from pandas.computation import expressions as expr

In [9]:
# set configure
path = "./CDR_CONCAT/"
filename = "CDR_CONCAT_TABLE_4"
relative_filename = path + filename + ".csv"
# out_filename = "mds_mly_minus20160901"
# after the numeric_st_idx's number will be tranform to int64
numeric_st_idx = 1
n_clusters = 5

In [10]:
# read revise csv file and print cost time
# just load 5 data 
t0 = time()
df = pd.read_csv(relative_filename, error_bad_lines=False, nrows = 5)
print("time for read csv: %.2f" % (time()-t0))

time for read csv: 0.03


In [11]:
df

,MINING_DW_SUBSCR_NO,VOICE_MO_HOUR_0_CNT,VOICE_MO_HOUR_1_CNT,VOICE_MO_HOUR_2_CNT,VOICE_MO_HOUR_3_CNT,VOICE_MO_HOUR_4_CNT,VOICE_MO_HOUR_5_CNT,VOICE_MO_HOUR_6_CNT,VOICE_MO_HOUR_7_CNT,VOICE_MO_HOUR_8_CNT,...,VOICE_MT_MON_TIME,VOICE_MT_TUE_TIME,VOICE_MT_WED_TIME,VOICE_MT_THU_TIME,VOICE_MT_FRI_TIME,VOICE_MT_SAT_TIME,workday_cnt.1,workday_time.1,holiday_cnt.1,holiday_time.1
0,11153,0,0,0,0,0,0,0,0,2,...,399,124,341,438,119,83,23,1421,3,154
1,11622,0,0,0,0,0,0,0,0,2,...,1869,427,595,1711,6031,1433,76,10633,25,1839
2,111054,0,0,0,0,0,0,0,0,0,...,146,133,14,219,116,310,18,628,18,544
3,111256,0,0,0,0,0,0,0,0,0,...,281,171,356,307,57,438,32,1172,26,769
4,111603,0,0,0,0,0,0,0,2,0,...,1781,311,121,329,189,241,45,2731,18,490


In [12]:
# use dask load full data
# Create the dataframe
# http://jcrist.github.io/dask-sklearn-part-3.html
cols = df.columns[numeric_st_idx:]
t0 = time()
# dask_df no columns slice, so u should do it in here, use usecols argument.
dask_df = dd.read_csv(relative_filename,
                 usecols=cols,
                 blocksize=int(128e6),
                 storage_options=dict(anon=True))
print("time for read csv: %.2f" % (time()-t0))

time for read csv: 0.04


In [13]:
t0 = time()
kmeans = KMeans(n_clusters=n_clusters).fit(dask_df)
print("time for k mean csv: %.2f " % (time()-t0))

time for k mean csv: 24.02 


In [22]:
count = 0
for row in dask_df.iterrows():
    count += 1
    if(count >10):
        break
    print(row)
#     print(kmeans.predict(row))

(0, VOICE_MO_HOUR_0_CNT          0
VOICE_MO_HOUR_1_CNT          0
VOICE_MO_HOUR_2_CNT          0
VOICE_MO_HOUR_3_CNT          0
VOICE_MO_HOUR_4_CNT          0
VOICE_MO_HOUR_5_CNT          0
VOICE_MO_HOUR_6_CNT          0
VOICE_MO_HOUR_7_CNT          0
VOICE_MO_HOUR_8_CNT          2
VOICE_MO_HOUR_9_CNT          0
VOICE_MO_HOUR_10_CNT         3
VOICE_MO_HOUR_11_CNT         7
VOICE_MO_HOUR_12_CNT         6
VOICE_MO_HOUR_13_CNT         8
VOICE_MO_HOUR_14_CNT        11
VOICE_MO_HOUR_15_CNT         5
VOICE_MO_HOUR_16_CNT         3
VOICE_MO_HOUR_17_CNT         2
VOICE_MO_HOUR_18_CNT         2
VOICE_MO_HOUR_19_CNT         1
VOICE_MO_HOUR_20_CNT         0
VOICE_MO_HOUR_21_CNT         1
VOICE_MO_HOUR_22_CNT         0
VOICE_MO_HOUR_23_CNT         0
VOICE_MO_HOUR_0_TIME         0
VOICE_MO_HOUR_1_TIME         0
VOICE_MO_HOUR_2_TIME         0
VOICE_MO_HOUR_3_TIME         0
VOICE_MO_HOUR_4_TIME         0
VOICE_MO_HOUR_5_TIME         0
                          ... 
VOICE_MO_HOUR_MON_TIME     367
VOIC

In [9]:
del dask_df

In [8]:
del df